## Testing the chroma db

That will be involving testing first everything locally: 

- data loaders
- splitting the data into chunks
- indexing the chunks into the vector store (chromadb in this case)
- creating a retriever
- setting up langchain retrievalQA chain 
- to do later: make it so that the bot remembers the conversation of the chain. (https://python.langchain.com/docs/expression_language/cookbook/retrieval)


In [1]:
# import libraries
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

In [4]:
# change directory to the root of the project
os.chdir('..')

In [3]:
FOLDER_PATH = 'raw-data/commented-penal-code'

In [5]:
# test what happens to the corrupted files
loader = PyPDFLoader(file_path='raw-data/CSJN/LibroVol308.1.pdf')

textSplitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
)

try: 
    data = loader.load_and_split(textSplitter)
except Exception as e:
    print(e)


EOF marker not found


Stream has ended unexpectedly


In [6]:
# test what happens to a correct file
loader = PyPDFLoader(file_path='raw-data/CSJN/LibroVol331-3-2008.pdf')

textSplitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
)

try: 
    data = loader.load_and_split(textSplitter)
except Exception as e:
    print(e)


In [8]:
type(data)

list

### Splittting the data into chunks!

In [ ]:
from langchain.document_loaders import DirectoryLoader

In [ ]:
textSplitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
)

In [ ]:
directoryLoader = DirectoryLoader(FOLDER_PATH, show_progress=True, use_multithreading=True, loader_cls=PyPDFLoader)

In [ ]:
documents = directoryLoader.load_and_split(textSplitter) # change this variable name to chunk next time

In [ ]:
len(documents)

### Embeddings Model

In [5]:
# using it now for embeddin the actual documents
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [6]:
# create the open-source embedding function
## here may be possible that it will be necessary to have to run the embeddings with the openai model
embeddingFunction = SentenceTransformerEmbeddings(model_name='sentence-transformers/LaBSE')

/opt/homebrew/anaconda3/envs/rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### indexing the chunks to chroma db

In [6]:
from langchain.vectorstores import Chroma

In [ ]:
chromaDatabase = Chroma.from_documents(documents, embeddingFunction, persist_directory='chromaDB')

### loading the index from disk

In [8]:
# load from disk
loadedChromaDatabase = Chroma(persist_directory='chromaDB', embedding_function=embeddingFunction)

In [10]:
answerDocs = loadedChromaDatabase.similarity_search_with_score('Delito')
answerDocs

[(Document(page_content='tenido la intención deliberada de no presentar la declaración jurada patrimonial en \nel momento oportuno. \nAdemás, cabe inferir la existencia de un especial elemento subjetivo distinto del \ndolo, cuyo contenido ha sido definido como la intención de inducir a error sobre la \nsituación patrimonial que debe reflejar la declaración jurada :>84. \nDebe destacarse la relevancia de las exigencias subjetivas referidas, ya que preci\xad\nsamente la malicia del sujeto caracterizará su omisión como delictiva y la diferencia\xad\nrá de una infracción meramente administrativa 585. Un aspecto importante a ser des\xad\ntacado consiste en la dificultad que puede presentar cada caso para acreditar que se \nha omitido maliciosamente. Una pauta que refleje ese ánimo en el sujeto puede en-\n(582) DO:-;NA, "Delitos ... ", p. 410. \n(583) En CNFed. Crim. v Corree., sala n, "Chescotta", del 2003/ 12/30, se descartó la posibi\xad\nlidad de imputar este delito\'en virtud de una omi

### testing both chromadb objects for a similiraty search

In [8]:
query = 'Que tipo de crimen es cuando alguien roba un celular?'
query2 = 'Que tipo de lesion es cuando alguien le pega a otra persona y le saca los dientes?'

In [12]:
# using the loaded from disk version

answerDocs = loadedChromaDatabase.similarity_search_with_score(query2)
bestRatedAnswer , score = answerDocs[0]
print(f'ANSWER with a score of {score}: \n')
print(bestRatedAnswer.page_content)
print('\nSOURCE: \n')
print(f'Found in {bestRatedAnswer.metadata["source"]} in page {bestRatedAnswer.metadata["page"]}')

ANSWER with a score of 1.12496018409729: 

tipo que comentamos del previsto en el inc. 1" del art. 143, ya que en esta última 
disposición el sujeto activo debe ser funcionario público y la privación ilegal de la 
libertad debe producirse con abuso de las funciones propias de aquel funcionario o 
con violación de las formalidades prescriptas por la ley 1.10 . 
. ~' 
Art. 142. -Se aplicará prisión o reclusión de dos a seis años, al que pri­
vare a otro de su libertad personal, cuando concurra alguna de las circuns­
tancias siguientes: 
1 ° Si el hecho se cometiere con violencias o amenazas o con fines reli­
giosos o de venganza; 
2° Si el hecho se cometiere en la persona de un ascendiente, de un her­
mano, del cónyuge o de otro individuo a quien se deba respeto particular; 
3° Si resultare grave daño a la persona, a la salud o a los negocios del 
ofendido, siempre que el hecho no importare otro delito por el cual la ley 
imponga pena mayor; 
4° Si el hecho se cometiere simulando autorid

### generating the retriever out of the chroma db

In [17]:
retrieverMMR = loadedChromaDatabase.as_retriever(search_type="mmr")
retriever = loadedChromaDatabase.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [18]:
# search type mmr
retrieverMMR.get_relevant_documents(query)

[Document(page_content='diga cuándo una única resolución que da sentido final a varios movimientos puede ser \nrelevada como una unidad por el tipo penal J.1. Tal factor es -entonces-la propia \nestructura del tipo delictivo en cada caso particular: aunque el factor final que rige un \nproceso causal sea el mismo (matar a alguien), alguno de los actos particulares realiza\xad\ndos puede tener, aisladamente, relevancia para distintos tipos delictivos (por ejemplo, \nla ilícita tenencia de un arma de fuego para el delito de portación ilegítima de armas de \nguerra) I~,. \nAsí, Zaffaroni, Alagia y Slokar sostienen que "el criterio de delimitación para la \ndeterminación de la consideración unitaria de I\'arios mOl\'Ímientos vinculados por el \nfactor final es tarea que incumbe a los tipos penales, debiendo extraerse del sentido de \nlos respectivos tipos penales en cuestión, tal como se obtiene mediante interpretación "lG. \nEstablecido -entonces- el concepto de unidad de hecho o acción (

In [19]:
# search type similarity
retriever.get_relevant_documents(query)

[Document(page_content='diga cuándo una única resolución que da sentido final a varios movimientos puede ser \nrelevada como una unidad por el tipo penal J.1. Tal factor es -entonces-la propia \nestructura del tipo delictivo en cada caso particular: aunque el factor final que rige un \nproceso causal sea el mismo (matar a alguien), alguno de los actos particulares realiza\xad\ndos puede tener, aisladamente, relevancia para distintos tipos delictivos (por ejemplo, \nla ilícita tenencia de un arma de fuego para el delito de portación ilegítima de armas de \nguerra) I~,. \nAsí, Zaffaroni, Alagia y Slokar sostienen que "el criterio de delimitación para la \ndeterminación de la consideración unitaria de I\'arios mOl\'Ímientos vinculados por el \nfactor final es tarea que incumbe a los tipos penales, debiendo extraerse del sentido de \nlos respectivos tipos penales en cuestión, tal como se obtiene mediante interpretación "lG. \nEstablecido -entonces- el concepto de unidad de hecho o acción (

### Setting up rag pipeline

In [20]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0, max_tokens=100)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    verbose=True,
)

chain.run(query2)

following this guide a more "complex" rag pipeline: https://python.langchain.com/docs/use_cases/question_answering/

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible. When giving an answer say where you found the answer(which document and which page).
Always say "gracias por preguntar!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""

rag_prompt_custom = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever , "question": RunnablePassthrough()}
    | rag_prompt_custom
    | llm
    | StrOutputParser()
)

rag_chain.invoke(query2)

In [ ]:
from operator import itemgetter
from langchain.schema.runnable import RunnableParallel

rag_chain_from_docs = (
    {
        "context": lambda input: input["documents"],
        "question": itemgetter("question"),
    }
    | rag_prompt_custom
    | llm
    | StrOutputParser()
)
rag_chain_with_source = RunnableParallel(
    {"documents": retriever, "question": RunnablePassthrough()}
) | {
    "documents": lambda input: [doc.metadata for doc in input["documents"]],
    "answer": rag_chain_from_docs,
}

rag_chain_with_source.invoke(query2)

In [ ]:
# streaming
for chunk in rag_chain_with_source.stream(query2):
    print(chunk, end="", flush=True)

### Adding memory to the chat

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

condense_q_system_prompt = """Given a chat history and the latest user question \
which might reference the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
condense_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", condense_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
condense_q_chain = condense_q_prompt | llm | StrOutputParser()

In [ ]:
from langchain.schema.messages import AIMessage, HumanMessage

condense_q_chain.invoke(
    {
        "chat_history": [
            HumanMessage(content="What does LLM stand for?"),
            AIMessage(content="Large language model"),
        ],
        "question": "What is meant by large",
    }
)

In [ ]:
qa_system_prompt = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible. When giving an answer ALWAYS say where you found the answer(DOCUMENT and PAGE found in the metadata).
{context}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)


def condense_question(input: dict):
    if input.get("chat_history"):
        return condense_q_chain
    else:
        return input["question"]


rag_chain = (
    RunnablePassthrough.assign(context=condense_question | retriever )
    | qa_prompt
    | llm
)

In [ ]:
chat_history = []

question = query2
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg])

second_question = "Cual es la pena para ese delito?"
ai_ms_2 = rag_chain.invoke({"question": second_question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=second_question), ai_ms_2])

In [ ]:
third_question = "Que numero de articulo es el delito mencionado?"
ai_ms_3 = rag_chain.invoke({"question": third_question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=third_question), ai_ms_3])

In [ ]:
chat_history

## Running chroma on a docker container

In [1]:
# create the chroma client
import chromadb
from chromadb.config import Settings
from langchain.vectorstores import Chroma

client = chromadb.HttpClient(host='localhost' , port=8000, settings=Settings(allow_reset=True))

In [29]:
client.reset()

True

In [21]:
# load a test documents from the penal code
loader = PyPDFLoader(file_path='/Users/nraffa/projects/juriBot/raw-data/commented-penal-code/D´alessio, Andrés J. - Codigo Penal Comentado y Anotado Parte Especial -Tomo II.pdf')

textSplitter = RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=0,
)

data = loader.load_and_split(textSplitter)

In [27]:
from chromadb.utils import embedding_functions

sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name='sentence-transformers/LaBSE')

In [ ]:
import uuid

# creating collection using custom embedding function from sentence transformer library
collection = client.create_collection(name="codigo_penal" , embedding_function=sentence_transformer_ef)

In [31]:
for doc in data:
    collection.add(
        ids=[str(uuid.uuid1())],
        metadatas=doc.metadata,
        documents=doc.page_content,
    )

In [4]:
# get all collections saved in the chroma db
client.get_collection("penal-code")

Collection(name=penal-code)

In [7]:
# tell LangChain to use our client and collection name
dockerChromaDB = Chroma(
    client=client,
    collection_name="penal-code",
    embedding_function=embeddingFunction,
)

In [13]:
collection = client.get_collection(name="commented-penal-code") # Get a collection object from an existing collection, by name. Will raise an exception if it's not found.

In [14]:
collection.peek()

{'ids': ['03eb5d6e-9b91-11ee-a23d-c67f47f74d7c',
  '04026e82-9b91-11ee-a23d-c67f47f74d7c',
  '040b6dd4-9b91-11ee-a23d-c67f47f74d7c',
  '041a96ec-9b91-11ee-a23d-c67f47f74d7c',
  '0426c37c-9b91-11ee-a23d-c67f47f74d7c',
  '0432d176-9b91-11ee-a23d-c67f47f74d7c',
  '043e91be-9b91-11ee-a23d-c67f47f74d7c',
  '044b106a-9b91-11ee-a23d-c67f47f74d7c',
  '0457c5c6-9b91-11ee-a23d-c67f47f74d7c',
  '0462e1fe-9b91-11ee-a23d-c67f47f74d7c'],
 'embeddings': [[-0.006641964428126812,
   -0.034019749611616135,
   -0.014707425609230995,
   -0.04475422203540802,
   -0.05360066518187523,
   -0.057412806898355484,
   -0.05107198655605316,
   -0.013145478442311287,
   0.035080406814813614,
   0.011024408973753452,
   -0.04011448472738266,
   -0.03764647990465164,
   -0.044714849442243576,
   0.031718529760837555,
   -0.053053077310323715,
   0.06075732782483101,
   0.0020697894506156445,
   0.024948474019765854,
   -0.03211723640561104,
   -0.01957477629184723,
   0.014734497293829918,
   -0.0423307828605175,
  

In [9]:
docs = dockerChromaDB.similarity_search("hola")

In [10]:
docs

[]